In [2]:
from logicqubit.logic import *
from cmath import *
import numpy as np
import sympy as sp
import scipy
from scipy.optimize import *
import matplotlib.pyplot as plt

Cuda is not available!
logicqubit version 1.5.8


https://arxiv.org/abs/1304.3061

https://cpb-us-w2.wpmucdn.com/voices.uchicago.edu/dist/0/2327/files/2020/10/SimultaneousMeasurementVQE.pdf1

In [3]:
gates = Gates(1)

ID = gates.ID()
X = gates.X()
Y = gates.Y()
Z = gates.Z()

In [4]:
II = ID.kron(ID)

XX = X.kron(X)
XI = X.kron(ID)
IX = ID.kron(X)
XY = X.kron(Y)
XZ = X.kron(Z)

YY = Y.kron(Y)
YI = Y.kron(ID)
IY = ID.kron(Y)
YX = Y.kron(X)
YZ = Y.kron(Z)

ZZ = Z.kron(Z)
ZI = Z.kron(ID)
IZ = ID.kron(Z)
ZX = Z.kron(X)
ZY = Z.kron(Y)

sig_is = np.kron([1, 1], [1, -1])
sig_si = np.kron([1, -1], [1, 1])
ZZ.get()

array([[ 1,  0,  0,  0],
       [ 0, -1,  0,  0],
       [ 0,  0, -1,  0],
       [ 0,  0,  0,  1]])

In [5]:
#H = XZ + ZI*2 # -ZX + 2*ZI

H = XX*3 + ZZ*7

min(scipy.linalg.eig(H.get())[0])

(-10+0j)

In [6]:
def ansatz(reg, params):
    n_qubits = len(reg)
    depth = n_qubits
    for i in range(depth):
        for j in range(n_qubits):
            if(j < n_qubits-1):
                reg[j+1].CNOT(reg[j])
            reg[i].RY(params[j])
            
def ansatz_2q(q1, q2, params):
    q2.CNOT(q1)
    q1.RY(params[0])
    q2.RY(params[1])
    q1.CNOT(q2)
    q1.RY(params[2])
    q2.RY(params[3])
    q2.CNOT(q1)
    q1.RY(params[4])
    q2.RY(params[5])
    q1.CNOT(q2)

In [7]:
def expectation_2q(params):
    logicQuBit  = LogicQuBit(2)
    q1 = Qubit()
    q2 = Qubit()

    ansatz_2q(q1,q2,params)
    psi = logicQuBit.getPsi()
    
    return (psi.adjoint()*H*psi).get()[0][0]

minimum = minimize(expectation_2q, [0,0,0,0,0,0], method='Nelder-Mead', options={'xtol': 1e-10, 'ftol': 1e-10})
print(minimum)

/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:586: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[k] = func(sim[k])
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:611: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxe
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:618: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxr
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:614: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxr
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:637: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxcc
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:627: ComplexWarning: Casting com

 final_simplex: (array([[ 5.31847022, -0.42327127, -3.1307354 ,  1.06588762,  1.01423211,
        -3.42077752],
       [ 5.31847022, -0.42327127, -3.1307354 ,  1.06588762,  1.01423211,
        -3.42077752],
       [ 5.31847022, -0.42327127, -3.1307354 ,  1.06588762,  1.01423211,
        -3.42077752],
       [ 5.31847022, -0.42327127, -3.1307354 ,  1.06588762,  1.01423211,
        -3.42077752],
       [ 5.31847022, -0.42327127, -3.1307354 ,  1.06588762,  1.01423211,
        -3.42077752],
       [ 5.31847022, -0.42327127, -3.1307354 ,  1.06588762,  1.01423211,
        -3.42077752],
       [ 5.31847022, -0.42327127, -3.1307354 ,  1.06588762,  1.01423211,
        -3.42077752]]), array([-10., -10., -10., -10., -10., -10., -10.]))
           fun: -10.000000000000004
       message: 'Optimization terminated successfully.'
          nfev: 1065
           nit: 614
        status: 0
       success: True
             x: array([ 5.31847022, -0.42327127, -3.1307354 ,  1.06588762,  1.01423211,
     

/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:644: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[j] = func(sim[j])


In [16]:
def expectation_value(measurements, base = np.array([1,-1,-1,1])):
    probabilities = np.array(measurements)
    expectation = np.sum(base * probabilities)
    return expectation

def sigma_xx(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
    
    # medidas em XX
    q1.RY(-pi/2)
    q2.RY(-pi/2)
    
    result = logicQuBit.Measure([q1,q2])
    result = expectation_value(result)
    return result

def sigma_yy(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
    
    # medidas em YY
    q1.RX(pi/2)
    q2.RX(pi/2)
    
    result = logicQuBit.Measure([q1,q2])
    yy = expectation_value(result)
    iy = expectation_value(result, sig_is)
    yi = expectation_value(result, sig_si)
    return yy, iy, yi

def sigma_zz(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
          
    result = logicQuBit.Measure([q1,q2])
    zz = expectation_value(result)
    iz = expectation_value(result, sig_is) # [zz, iz] = 0
    zi = expectation_value(result, sig_si) # [zz, zi] = 0
    return zz, iz, zi

def sigma_zx(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
    
    # medidas em X
    #q1.RY(-pi/2)
    q2.RY(-pi/2)
    
    result = logicQuBit.Measure([q1,q2])
    zx = expectation_value(result)
    return zx

def sigma_xx_zz(params):
    logicQuBit  = LogicQuBit(2)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
    
    # medida na base de bell
    q2.CX(q1)
    q1.H()    # Hx1
    
    result = logicQuBit.Measure([q1,q2])
    zi = expectation_value(result, sig_si)
    iz = expectation_value(result, sig_is)
    xx = zi
    zz = iz
    return xx, zz

def expectation_energy(params):
    #xx =  sigma_xx(params)
    #yy =  sigma_yy(params)
    
    #zz, iz, zi = sigma_zz(params)
    #zx = sigma_zx(params)   
    #result = -zx + zi*2
    
    xx, zz = sigma_xx_zz(params) 
    result = xx*3 + zz*7
    return result

In [9]:
minimum = minimize(expectation_energy, [0,0,0,0,0,0], method='Nelder-Mead', options={'xtol': 1e-10, 'ftol': 1e-10})
print(minimum)

 final_simplex: (array([[ 2.19396352, -0.53207923,  0.91245827, -3.03402174,  0.31749551,
        -1.89342844],
       [ 2.19396352, -0.53207923,  0.91245827, -3.03402174,  0.31749551,
        -1.89342844],
       [ 2.19396352, -0.53207923,  0.91245827, -3.03402174,  0.31749551,
        -1.89342844],
       [ 2.19396352, -0.53207923,  0.91245827, -3.03402174,  0.31749551,
        -1.89342844],
       [ 2.19396352, -0.53207923,  0.91245828, -3.03402174,  0.31749551,
        -1.89342844],
       [ 2.19396352, -0.53207923,  0.91245827, -3.03402174,  0.31749551,
        -1.89342844],
       [ 2.19396352, -0.53207923,  0.91245827, -3.03402174,  0.31749551,
        -1.89342844]]), array([-10., -10., -10., -10., -10., -10., -10.]))
           fun: -10.000000000000005
       message: 'Optimization terminated successfully.'
          nfev: 932
           nit: 543
        status: 0
       success: True
             x: array([ 2.19396352, -0.53207923,  0.91245827, -3.03402174,  0.31749551,
      

In [10]:
def gradient(params, evaluate, shift = pi/2):
    n_params = params.shape[0]
    gradients = np.zeros(n_params)
    
    for i in range(n_params):
        #parameter shift rule
        shift_vect = np.array([shift if j==i else 0 for j in range(n_params)])
        shift_right = params + shift_vect
        shift_left = params - shift_vect
        
        expectation_right = evaluate(shift_right)
        expectation_left = evaluate(shift_left)

        gradients[i] = expectation_right - expectation_left

    return gradients

In [11]:
params = np.random.uniform(-np.pi, np.pi, 6)
last_params = np.zeros(6)

In [12]:
lr = 0.1
err = 1
while err > 1e-3:
    grad = gradient(params, expectation_energy, pi/1.3)
    params = params - lr*grad
    err = abs(sum(params - last_params))
    last_params = np.array(params)
    print(err) 

2.740998722971736
0.7664059946733903
2.6551149987384
1.2608214802431843
1.6463140203823585
0.9882522116289967
0.7597045148619697
0.7288509454263662
0.48616133345178747
0.5674605061569935
0.38406516697773163
0.43028564541804637
0.29331113347599436
0.3180409806639399
0.22291660083314208
0.23564204761260749


/tmp/ipykernel_13066/2399136664.py:14: ComplexWarning: Casting complex values to real discards the imaginary part
  gradients[i] = expectation_right - expectation_left


0.17091061131625096
0.17690361978602842
0.13286436551320124
0.1350978736282606
0.1047919107121954
0.10491825192171336
0.0837058159106186
0.08266834303990159
0.06753875093640191
0.06589648464516769
0.05490283477970613
0.0529959455511186
0.04486783316222952
0.04290418378493158
0.03679987111384837
0.03490346410358293
0.030255093195600533
0.02849544969493395
0.024912658187603715
0.023323817622205006
0.0205332435402516
0.019126519558116256
0.016933316866161174
0.015705975012091533
0.013968936519843905
0.012910073278434617
0.011525275889824105
0.010619738007284787
0.009509607723144992
0.008740537754570302
0.007846417010252585
0.0071968454167906926
0.006473860085199415
0.005927646503809669
0.005341106487352892
0.004883446257502394
0.004406284195538374
0.004023932295785418
0.0036348551808148
0.0033161729228793763
0.002998310149989969
0.002733206203465899
0.0024731080036946296
0.0022529213447672614
0.002039807784688666
0.0018571634170972545
0.00168235488315098
0.0015310116825909825
0.0013874924

In [13]:
expectation_energy(params)

(-9.999986367379375+0j)